In [1]:
from functional import seq
from functional.pipeline import Sequence
from typing import Iterator, NamedTuple
from tia.cov import FileTableRow, ContextTableRow, LineTableRow, get_context_table, get_file_table, get_line_table
from tia.maps import CoverageMap, CoverageMapSingle, ImpactMap, ImpactMapSingle

In [2]:
db_path = '../tests/data/.coverage'

In [3]:
file_table = get_file_table(db_path)
#assert type(file_table) == Sequence

In [4]:
context_table = get_context_table(db_path)
#assert type(context_table) == Sequence

In [5]:
line_table = get_line_table(db_path)
#assert type(line_table) == Sequence

In [6]:
def get_impact_map(file_table_rows, line_table_rows, context_table_rows, file_paths) -> ImpactMap:
    """
    For every file_path:
        file_path -> file_table_rows -> file_id -> line_table_row -> tests
    """
    impact_map = []  # TODO: get rid of temporary hashable list for later sequence generation
    # explicit caching required, otherwise raise of "IndexError: list index out of range"
    file_table_rows.cache()
    line_table_rows.cache()
    context_table_rows.cache()
    for file_path in file_paths:
        # TODO: make search strict and fuzzy search in separate function (SRP)
        file = file_table_rows.filter(lambda x: file_path in x.path)  # fuzzy search like filtering
        file_id = file.head().file_id
        impacted_lines_rows = line_table_rows.filter(lambda x: x.file_id == file_id)
        test_ids = (l.context_id for l in impacted_lines_rows)
        tests = set()
        for test_id in test_ids:
            impacted_context_rows = context_table_rows.filter(lambda x: x.context_id == test_id)
            test = impacted_context_rows.head().context
            # workaround for coveragepy 5.02a empty and irrelevant context entries 
            if test is not "" or not "testsfailed":
                tests.add(test)
        filter(None, tests)  # get rid of empty set element
        impact_map.append(ImpactMapSingle(file_path, seq(tests)))
        del tests  # free memory from set
    return seq(impact_map)
    del impact_map  # free memory from list

In [7]:
changed_production_files = ['tia/config.py', 'tia/env.py']
impacted_tests = get_impact_map(file_table, line_table, context_table, changed_production_files)
assert type(impacted_tests) == Sequence
impacted_tests

production_code,tests
tia/config.py,"['test_read_valid_single_pipeline_with_dirs_only_config', 'test_reading_existing_valid_config_file_returns_string', 'test_read_valid_single_pipeline_with_files_only_config', 'test_read_invalid_pipelines_config', 'test_reading_existing_invalid_config_file_raises_error', 'test_read_valid_parent_key_config', 'test_read_valid_explicit_full_blown_pipelines_config', 'test_reading_non_existing_config_file_raises_exception', 'test_read_valid_implicit_full_blown_pipelines_config']"
tia/env.py,"['test_is_some_ci', 'test_is_no_ci']"


In [8]:
def get_coverage_map(context_table_rows, line_table_rows, file_table_rows, tests) -> CoverageMap:
    """
    For every test:
        test aka context -> context_table_rows -> context_id -> line_table_row -> file_paths
    """
    coverage_map = []  # TODO: get rid of temporary hashable list for later sequence generation
    # explicit caching required, otherwise raise of "IndexError: list index out of range"
    context_table_rows.cache()
    line_table_rows.cache()
    file_table_rows.cache()
    for test_name in tests:
        # TODO: make search strict and fuzzy search in separate function (SRP)
        test = context_table_rows.filter(lambda x: test_name in x.context)  # fuzzy search like filtering
        test_id = test.head().context_id
        covered_lines_rows = line_table_rows.filter(lambda x: x.context_id == test_id)
        file_ids = (l.file_id for l in covered_lines_rows)
        file_paths = set()
        for file_id in file_ids:
            covered_file_rows = file_table_rows.filter(lambda x: x.file_id == file_id)
            file_path = covered_file_rows.head().path
            file_paths.add(file_path)        
        coverage_map.append(CoverageMapSingle(test_name, seq(file_paths)))
        del file_paths  # free memory from set
    return seq(coverage_map)
    del coverage_map  # free memory from list

In [9]:
changed_tests = ['test_read_valid_parent_key_config', 'test_read_invalid_pipelines_config', 'test_is_no_ci']
# explicit caching required here if context_table and line_table are not cached before, otherwise get_coverage_map raises "IndexError: list index out of range"
impacted_production_files = get_coverage_map(context_table, line_table, file_table, changed_tests)
assert type(impacted_production_files) == Sequence
impacted_production_files

test,production_code
test_read_valid_parent_key_config,['/home/fk/github/python-tia/tia/config.py']
test_read_invalid_pipelines_config,['/home/fk/github/python-tia/tia/config.py']
test_is_no_ci,['/home/fk/github/python-tia/tia/env.py']


In [10]:
from typing import NamedTuple

FilePath = str

Line = int
Lines = Iterator[Line]

class FileCoverage(NamedTuple):
    """
    File based line coverage.
    """
    production_code: FilePath
    covered_lines: Lines

FileCoverages = Iterator[FileCoverage]

In [11]:
def get_covered_lines(file_table_rows, line_table_rows, file_paths) -> FileCoverages:
    """
    For every file_path:
        file_path -> file_table -> file_id -> line_table -> covered_lines
    """
    file_coverages = []
    # explicit caching required, otherwise raise of "IndexError: list index out of range"
    file_table_rows.cache()
    line_table_rows.cache()
    for file_path in file_paths:
        covered_lines = []
        file = file_table_rows.filter(lambda x: file_path in x.path)  # fuzzy search like filtering
        file_id = file.head().file_id
        covered_lines_rows = line_table_rows.filter(lambda x: x.file_id == file_id)
        covered_lines = [l.lineno for l in covered_lines_rows]
        file_coverages.append(FileCoverage(file_path, seq(covered_lines)))
        del covered_lines
    return seq(file_coverages)
    del file_coverages  # free memory from list

In [12]:
production_files = ['tia/__init__.py', 'tia/config.py', 'tia/env.py']
# explicit caching required here if file_table and line_table are not cached before, otherwise get_covered_lines raises "IndexError: list index out of range"
file_coverages = get_covered_lines(file_table, line_table, production_files)
assert type(file_coverages) == Sequence
file_coverages

production_code,covered_lines
tia/__init__.py,[1]
tia/config.py,"[1, 2, 3, 5, 8, 9, 12, 25, 30, 17, 18, 20, 22, 17, 18, 20, 21, 17, 18, 19, 26, 27, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 51, 52, 53, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 51, 52, 53, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 51, 52, 53, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 51, 52, 53, 26, 27, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45, 46, 47, 51, 52, 54, 55]"
tia/env.py,"[7, 10, 20, 27, 34, 41, 48, 55, 62, 69, 76, 83, 90, 12, 23, 24, 91, 92, 15, 94, 30, 31, 37, 38, 44, 45, 51, 52, 13, 58, 59, 65, 66, 72, 73, 79, 80, 14, 86, 87, 12, 23, 24, 91, 94, 30, 31, 37, 38, 44, 45, 51, 52, 13, 58, 59, 65, 66, 72, 73, 79, 80, 14, 86, 87, 17]"
